# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [5]:
data.shape

(68584, 7)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [59]:
by_customer = data.groupby(by=['CustomerID','ProductName']).sum()[['Quantity']]
by_customer

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [23]:
pivot = pd.pivot_table(data, index='ProductName', columns='CustomerID', values= 'Quantity', fill_value=0)
pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


In [27]:
pivot.index

Index(['Anchovy Paste - 56 G Tube', 'Appetizer - Mini Egg Roll, Shrimp',
       'Appetizer - Mushroom Tart', 'Appetizer - Sausage Rolls',
       'Apricots - Dried', 'Apricots - Halves', 'Apricots Fresh',
       'Arizona - Green Tea', 'Artichokes - Jerusalem', 'Assorted Desserts',
       ...
       'Wine - White, Colubia Cresh', 'Wine - White, Mosel Gold',
       'Wine - White, Schroder And Schyl', 'Wine - Wyndham Estate Bin 777',
       'Wonton Wrappers', 'Yeast Dry - Fermipan', 'Yoghurt Tubes',
       'Yogurt - Blueberry, 175 Gr', 'Yogurt - French Vanilla',
       'Zucchini - Yellow'],
      dtype='object', name='ProductName', length=452)

In [28]:
pivot.columns

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

In [30]:
pivot.loc['Anchovy Paste - 56 G Tube',639]

1

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [40]:
pivot_customer = pd.pivot_table(data, 
                                 index='CustomerID', 
                                 columns='ProductName', 
                                 values= 'Quantity', 
                                 fill_value=0)

In [45]:
df = pd.DataFrame(squareform(pdist(pivot_customer)),
                index=pivot_customer.index,
                columns=pivot_customer.index)
df.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,10.723805,9.643651,9.899495,10.440307,10.148892,10.723805,10.246951,10.344080,10.148892,...,206.963765,194.766527,199.145675,183.463893,202.259734,198.075743,203.921554,197.949489,218.824130,203.798921
200,10.723805,0.000000,10.677078,10.723805,11.489125,11.045361,10.770330,10.954451,10.677078,10.488088,...,206.269242,194.285872,198.423789,182.543146,202.044550,197.729613,202.723950,197.349943,218.625250,203.953426
264,9.643651,10.677078,0.000000,10.344080,10.198039,10.488088,10.677078,10.583005,10.488088,10.488088,...,206.366179,194.517351,199.153207,182.789496,201.772149,196.817174,203.315026,197.324606,218.029814,203.806281
356,9.899495,10.723805,10.344080,0.000000,10.630146,10.535654,10.630146,9.949874,10.630146,10.440307,...,206.726873,194.643263,199.025124,182.923481,202.264678,197.575302,204.293906,197.322072,218.142155,203.681123
412,10.440307,11.489125,10.198039,10.630146,0.000000,10.677078,10.677078,10.198039,10.770330,10.392305,...,206.985507,193.889144,199.168271,182.805908,202.034650,197.339808,204.066166,197.466453,217.928887,202.960587


In [49]:
similarity_index = 1/(1 + df)
similarity_index

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [63]:
# choosing customer #33: 
similarity_index[[33]].drop(33).sort_values(by=33, ascending=False).head(5)

CustomerID,33
CustomerID,
3909,0.095358
264,0.093953
3531,0.093953
2503,0.093498
3305,0.093051


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [102]:
similar_customers = [i for i in similarity_index[[33]].drop(33).sort_values(by=33, ascending=False).head(5).index]
similar_customers

[3909, 264, 3531, 2503, 3305]

In [104]:
recom_products = pd.concat([(by_customer.loc[customer]) for customer in similar_customers ])
recom_products

,Quantity
ProductName,
Bay Leaf,2
"Beans - Kidney, Canned",1
Beer - Original Organic Lager,1
Beer - Rickards Red,1
Black Currants,1
...,...
Wine - Ej Gallo Sierra Valley,2
Wine - Pinot Noir Latour,1
Wine - Redchard Merritt,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [85]:
products.groupby(by='ProductName').sum().sort_values(by='Quantity', ascending=False)

,Quantity
ProductName,
"Salsify, Organic",4
Quiche Assorted,3
Chocolate - Dark,3
Wine - Charddonnay Errazuriz,3
Bay Leaf,3
...,...
Gatorade - Xfactor Berry,1
Gloves - Goldtouch Disposable,1
Grouper - Fresh,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [87]:
target_cust_product = by_customer.loc[33]

In [96]:
recom_df = pd.merge (left=recom_products, right=target_cust_product, on='ProductName',how='left', suffixes=('_recom', '_trgt')).fillna(0)
recom_df

,Quantity_recom,Quantity_trgt
ProductName,,
Bay Leaf,2,0.0
"Beans - Kidney, Canned",1,1.0
Beer - Original Organic Lager,1,1.0
Beer - Rickards Red,1,1.0
Black Currants,1,1.0
...,...,...
Wine - Ej Gallo Sierra Valley,2,0.0
Wine - Pinot Noir Latour,1,0.0
Wine - Redchard Merritt,1,0.0


In [109]:
top5_prod = recom_df.query('Quantity_trgt==0').sort_values(by='Quantity_recom', ascending=False).head(5)
top5_prod 

,Quantity_recom,Quantity_trgt
ProductName,,
Bay Leaf,2,0.0
Sunflower Seed Raw,2,0.0
Butter - Unsalted,2,0.0
Wine - Charddonnay Errazuriz,2,0.0
Bar - Granola Trail Mix Fruit Nut,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [100]:
recomm_dict = {}

In [101]:
# list of all customers
all_customer = pivot_customer.index
all_customer

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

In [120]:
# list top 5 similar customers:

for customer in all_customer:
    similar_customers = [i for i in similarity_index[[customer]].drop(customer).sort_values(by=customer, ascending=False).head(5).index]
    recom_products = pd.concat([(by_customer.loc[customer]) for customer in similar_customers ])
    target_cust_product = by_customer.loc[customer]
    recom_df = pd.merge(left=recom_products, 
                        right = target_cust_product,
                                on='ProductName',
                                how='left', 
                                suffixes=('_recom', '_trgt')).fillna(0)
    top5_prod = recom_df.query('Quantity_trgt==0').sort_values(by='Quantity_recom', 
                                                               ascending=False).head(5)
    recomm_dict[customer] = list(top5_prod.index)

##  Step 9: Store the results in a Pandas data frame. The data frame should have a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [137]:
final_df = pd.DataFrame(recomm_dict).T
final_df['CustomerID'] = final_df.index
final_df = final_df[['CustomerID',0,1,2,3,4]].reset_index(drop=True)
final_df.columns = ['CustomerID', 'Top_1','Top_2','Top_3','Top_4','Top_5']
final_df

,CustomerID,Top_1,Top_2,Top_3,Top_4,Top_5
0,33,Bay Leaf,Sunflower Seed Raw,Butter - Unsalted,Wine - Charddonnay Errazuriz,Bar - Granola Trail Mix Fruit Nut
1,200,Pork - Kidney,Wine - Vidal Icewine Magnotta,Ice Cream Bar - Hageen Daz To,Tia Maria,Muffin - Carrot Individual Wrap
2,264,Pork - Kidney,Cheese - Parmesan Grated,"Salsify, Organic",Sage - Ground,"Veal - Inside, Choice"
3,356,Beans - Wax,Pasta - Angel Hair,"Pepper - Black, Whole",Wine - Vidal Icewine Magnotta,Knife Plastic - White
4,412,Tia Maria,Table Cloth 81x81 White,Pomello,Beets - Mini Golden,Wine - Vidal Icewine Magnotta
...,...,...,...,...,...,...
995,97928,Pomello,Wine - Vidal Icewine Magnotta,Water - Spring Water 500ml,"Appetizer - Mini Egg Roll, Shrimp",Ecolab - Lime - A - Way 4/4 L
996,98069,Soupfoamcont12oz 112con,Beans - Kidney White,Milk - 1%,Yoghurt Tubes,Kiwi
997,98159,"Wine - Red, Harrow Estates, Cab","Water, Tap",Flour - Whole Wheat,Onion Powder,Jolt Cola - Electric Blue
998,98185,Crackers - Trio,Beef - Montreal Smoked Brisket,"Cheese - Brie, Triple Creme",Cheese - Taleggio D.o.p.,Sugar - Fine


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [138]:
# Choosing different distance metric: 
dist_metric = 'cityblock'
recomm_dict = {}

df = pd.DataFrame(squareform(pdist(pivot_customer, metric=dist_metric)), 
                  index=pivot_customer.index,
                columns=pivot_customer.index)
similarity_index = 1/(1 + df)

for customer in all_customer:
    similar_customers = [i for i in similarity_index[[customer]].drop(customer).sort_values(by=customer, ascending=False).head(5).index]
    recom_products = pd.concat([(by_customer.loc[customer]) for customer in similar_customers ])
    target_cust_product = by_customer.loc[customer]
    recom_df = pd.merge(left=recom_products, 
                        right = target_cust_product,
                                on='ProductName',
                                how='left', 
                                suffixes=('_recom', '_trgt')).fillna(0)
    top5_prod = recom_df.query('Quantity_trgt==0').sort_values(by='Quantity_recom', 
                                                               ascending=False).head(5)
    recomm_dict[customer] = list(top5_prod.index)

In [140]:
final_df = pd.DataFrame(recomm_dict).T
final_df['CustomerID'] = final_df.index
final_df = final_df[['CustomerID',0,1,2,3,4]].reset_index(drop=True)
final_df.columns = ['CustomerID', 'Top_1','Top_2','Top_3','Top_4','Top_5']
final_df

,CustomerID,Top_1,Top_2,Top_3,Top_4,Top_5
0,33,Bay Leaf,Sunflower Seed Raw,Butter - Unsalted,Wine - Charddonnay Errazuriz,Bar - Granola Trail Mix Fruit Nut
1,200,Pork - Kidney,Ice Cream Bar - Oreo Cone,Ice Cream Bar - Hageen Daz To,"Oranges - Navel, 72",Wine - Chardonnay South
2,264,Pork - Kidney,Cheese - Parmesan Grated,V8 - Berry Blend,Cheese - Bocconcini,Muffin - Carrot Individual Wrap
3,356,Beans - Wax,Pasta - Angel Hair,"Pepper - Black, Whole",Wine - Vidal Icewine Magnotta,Knife Plastic - White
4,412,Pasta - Angel Hair,Wine - Vidal Icewine Magnotta,Tomato - Tricolor Cherry,Cheese - Bocconcini,Pomello
...,...,...,...,...,...,...
995,97928,"Salsify, Organic",Muffin - Carrot Individual Wrap,V8 - Berry Blend,Wine - Chardonnay South,"Mushrooms - Black, Dried"
996,98069,Wine - Chardonnay South,Butter - Unsalted,Rabbit - Whole,"Rum - Coconut, Malibu",Muffin - Carrot Individual Wrap
997,98159,Beans - Wax,Muffin - Carrot Individual Wrap,Bandage - Flexible Neon,Wine - Sogrape Mateus Rose,Curry Paste - Madras
998,98185,Pork - Kidney,Soup - Campbells Tomato Ravioli,"Nut - Pistachio, Shelled",Tia Maria,Pants Custom Dry Clean
